# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d7ebe82f8a-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("Trademarks across the world") 

Consider the following exploratory information need:

> You are investigating different kinds of trademarks

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wdt:P27`      | citizenship   | predicate |
| `wd:Q167270`  | trademark     | node      |
| `wd:Q14091`   | iMac          | node      |
| `wd:Q312`     | Apple Inc.    | node      |



Also consider

```
?p wdt:P31/wdt:P279* wd:Q167270  . 
```

is the BGP to retrieve all **entities that are trademarks**

## Workload Goals

1. Identify the BGP for the company or person related to a given trademark

2. Identify the BGP to retrieve types or categories for a given trademark

3. What companies have the largest number of trademarks? 

4. What are the types of trademarks, how many trademarks exist for each type?

5. Analyze the number of trademarks across types, companies, and countries
 
   5.1 How many U.S. companies hold trademarks? In which sectors?
   
   5.2 How many people hold or are connected to trademarks? In which role?
   
   5.3 In which sector there is the highest number of trademarks?   


In [1]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT( DISTINCT ?p)
WHERE { 


?p wdt:P31/wdt:P279* wd:Q167270  . 

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '25105')]


1

## Companies or people related to a trademark
This section contains the queries used to answer the first question: _Identify the BGP for the company or person related to a given trademark_

Firstly, I count how many properties have a trademark as subject or object.

In [3]:
queryString = """
SELECT COUNT(DISTINCT ?p) AS ?subjProps
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270;
    ?p ?obj. 

} 
"""

print("Results")
run_query(queryString)

Results
[('subjProps', '1197')]


1

In [4]:
queryString = """
SELECT COUNT(DISTINCT ?p) AS ?objProps
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270.
?s ?p ?t.

} 
"""

print("Results")
run_query(queryString)

Results
[('objProps', '226')]


1

Since subject properties are a lot I try to look for the properties where the object is a trademark. It is also reasonable that the connection between companies or people and trademarks have companies as subjects. I now list all these properties, limiting the results since I assume most trademarks have a property that links them to a company or person.

In [5]:
queryString = """
SELECT DISTINCT ?p ?pname COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270.

?s ?p ?t.

?p <http://schema.org/name> ?pname.

} 
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P264'), ('pname', 'record label'), ('trademarks', '7994')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('trademarks', '1491')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics'), ('trademarks', '1403')]
[('p', 'http://www.wikidata.org/prop/direct/P355'), ('pname', 'subsidiary'), ('trademarks', '1396')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pname', 'owner of'), ('trademarks', '1033')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of'), ('trademarks', '1002')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic"), ('trademarks', '982')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer'), ('trademarks', '599')]
[('p', 'http://www.wikidata.org/prop/direct/P199'), ('pname', 'business division'), ('trademarks', '537')]
[('p', 'http://www.wikidata.org/prop/direct/P749'), (

25

Interesting properties retrieved:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1716`    | brand    | predicate | 
| `wdt:P1830`    | owner of    | predicate | 

I recall there are 25105 trademarks in the database however these properties are present for at most 1000 trademarks therefore I think there are some other properties in the other direction that identify the required connection.

Before moving on to the other direction I check the retrieved properties to see if they are of any use. Firstly I retrieve some triples with the property "owner of" (wdt:P1830) to see what kind of information it containts.

In [2]:
queryString = """
SELECT DISTINCT ?s ?sname ?t ?trademark
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270.

?s wdt:P1830 ?t.

?s <http://schema.org/name> ?sname.
?t <http://schema.org/name> ?trademark.

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q720479'), ('sname', 'Asahi Breweries'), ('t', 'http://www.wikidata.org/entity/Q153075'), ('trademark', 'Grolsch Brewery')]
[('s', 'http://www.wikidata.org/entity/Q255363'), ('sname', 'Starship Entertainment'), ('t', 'http://www.wikidata.org/entity/Q12620832'), ('trademark', 'King Kong Entertainment')]
[('s', 'http://www.wikidata.org/entity/Q484449'), ('sname', 'Cube Entertainment'), ('t', 'http://www.wikidata.org/entity/Q21554828'), ('trademark', 'Play M Entertainment')]
[('s', 'http://www.wikidata.org/entity/Q625975'), ('sname', 'Kakao M'), ('t', 'http://www.wikidata.org/entity/Q21554828'), ('trademark', 'Play M Entertainment')]
[('s', 'http://www.wikidata.org/entity/Q625975'), ('sname', 'Kakao M'), ('t', 'http://www.wikidata.org/entity/Q67200500'), ('trademark', 'Cre.Ker Entertainment')]
[('s', 'http://www.wikidata.org/entity/Q95989661'), ('sname', 'Vesa Equity Investment'), ('t', 'http://www.wikidata.org/entity/Q63335'), ('trademark', 

25

From this first query it looks like "owner of" (wdt:P1830) contains information about which companies or people own a trademark. I recall there are 25105 trademarks in the database but such property is present only for 1033 of them therefore there must be other properties containing such information. 

I now look for some triples with the property "brand" (wdt:P1716).

In [6]:
queryString = """
SELECT DISTINCT ?s ?sname ?t ?trademark
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270.

?s wdt:P1716 ?t.

?s <http://schema.org/name> ?sname.
?t <http://schema.org/name> ?trademark.

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q28807997'), ('sname', 'Procter & Gamble - Rakona'), ('t', 'http://www.wikidata.org/entity/Q503592'), ('trademark', 'Gillette')]
[('s', 'http://www.wikidata.org/entity/Q15849235'), ('sname', 'Steyr 540'), ('t', 'http://www.wikidata.org/entity/Q98636365'), ('trademark', 'Steyr')]
[('s', 'http://www.wikidata.org/entity/Q15849239'), ('sname', 'Steyr 658'), ('t', 'http://www.wikidata.org/entity/Q98636365'), ('trademark', 'Steyr')]
[('s', 'http://www.wikidata.org/entity/Q117749'), ('sname', 'Hepatitis B vaccine'), ('t', 'http://www.wikidata.org/entity/Q17066597'), ('trademark', 'Twinrix')]
[('s', 'http://www.wikidata.org/entity/Q3553093'), ('sname', 'hepatitis A vaccine'), ('t', 'http://www.wikidata.org/entity/Q17066597'), ('trademark', 'Twinrix')]
[('s', 'http://www.wikidata.org/entity/Q18124882'), ('sname', 'Ford 3000'), ('t', 'http://www.wikidata.org/entity/Q98579722'), ('trademark', 'Ford')]
[('s', 'http://www.wikidata.org/entity/Q30097763'

25

Such property connects products to its brand (that usually is a trademark) therefore it gives no information about the companies or people related to it. 

Now I can look for properties in the other direction (i.e., where trademark is a subject). As I saw before there are 1197 such properties however I can return them based on the number of subjects that have the specific property. Since I am looking for a property that connect each trademark to its company or person it is reasonable to think that such property would have lots of subjects connected through it.

In [7]:
queryString = """
SELECT ?p ?pname COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270;
    ?p ?o.

?p <http://schema.org/name> ?pname.

}
GROUP BY ?p ?pname
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('trademarks', '25105')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('trademarks', '14653')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pname', 'inception'), ('trademarks', '12965')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('pname', 'official website'), ('trademarks', '11010')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pname', 'Freebase ID'), ('trademarks', '10601')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pname', 'Google Knowledge Graph ID'), ('trademarks', '6844')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location'), ('trademarks', '6650')]
[('p', 'http://www.wikidata.org/prop/direct/P966'), ('pname', 'MusicBrainz label ID'), ('trademarks', '5972')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('pname', 'Facebook ID'), ('trademarks', '3898')]
[('p', 'http://www.wikidata.org/prop/

25

By means of this query I retrieved some useful properties both for this task and for the next questions. Here I report the most interesting properties.

| IRI           | Description   | Number of Trademarks   |
| -----------   | -----------   |-----------|
| `wdt:P127`   | owned by   | 2620 |
| `wdt:P749`     | parent organization   | 2018 |
| `wdt:P136`    | genre    | 1951 | 
| `wdt:P452`    | industry   | 3730 |
| `wdt:P373`    | common category   | 3511 |
| `wdt:P17`    | country    | 14653 | 
| `wdt:P571`    | inception   | 12965 | 

I now check some triples of the properties "owned by" (wdt:P127) and "parent organization" (wdt:P749) while I store the other properties for later usage.

In [8]:
queryString = """
SELECT DISTINCT ?t ?trademark ?o ?owner
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270;
    wdt:P127 ?o.

?t <http://schema.org/name> ?trademark.
?o <http://schema.org/name> ?owner.

}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('t', 'http://www.wikidata.org/entity/Q18161548'), ('trademark', 'Network NorthWest'), ('o', 'http://www.wikidata.org/entity/Q7309132'), ('owner', 'Regional Railways')]
[('t', 'http://www.wikidata.org/entity/Q18161548'), ('trademark', 'Network NorthWest'), ('o', 'http://www.wikidata.org/entity/Q605169'), ('owner', 'British Rail')]
[('t', 'http://www.wikidata.org/entity/Q2535134'), ('trademark', 'Vue Entertainment'), ('o', 'http://www.wikidata.org/entity/Q3347600'), ('owner', 'OMERS')]
[('t', 'http://www.wikidata.org/entity/Q21554828'), ('trademark', 'Play M Entertainment'), ('o', 'http://www.wikidata.org/entity/Q484449'), ('owner', 'Cube Entertainment')]
[('t', 'http://www.wikidata.org/entity/Q21554828'), ('trademark', 'Play M Entertainment'), ('o', 'http://www.wikidata.org/entity/Q625975'), ('owner', 'Kakao M')]
[('t', 'http://www.wikidata.org/entity/Q484449'), ('trademark', 'Cube Entertainment'), ('o', 'http://www.wikidata.org/entity/Q7509507'), ('owner', 'SidusHQ')]
[('t', 

25

In [9]:
queryString = """
SELECT DISTINCT ?t ?trademark ?o ?owner
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270;
    wdt:P749 ?o.

?t <http://schema.org/name> ?trademark.
?o <http://schema.org/name> ?owner.

}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('t', 'http://www.wikidata.org/entity/Q18161548'), ('trademark', 'Network NorthWest'), ('o', 'http://www.wikidata.org/entity/Q605169'), ('owner', 'British Rail')]
[('t', 'http://www.wikidata.org/entity/Q12581039'), ('trademark', 'FNC Entertainment'), ('o', 'http://www.wikidata.org/entity/Q10854462'), ('owner', 'CJ E&M')]
[('t', 'http://www.wikidata.org/entity/Q255363'), ('trademark', 'Starship Entertainment'), ('o', 'http://www.wikidata.org/entity/Q625975'), ('owner', 'Kakao M')]
[('t', 'http://www.wikidata.org/entity/Q2756503'), ('trademark', 'Gaumont-British Picture Corporation'), ('o', 'http://www.wikidata.org/entity/Q913462'), ('owner', 'Gaumont Film Company')]
[('t', 'http://www.wikidata.org/entity/Q45282'), ('trademark', 'Pledis Entertainment'), ('o', 'http://www.wikidata.org/entity/Q16161254'), ('owner', 'Hybe Corporation')]
[('t', 'http://www.wikidata.org/entity/Q491998'), ('trademark', 'TS Entertainment'), ('o', 'http://www.wikidata.org/entity/Q10854462'), ('owner', '

25

By means of these two queries I can assess both properties retrieve information about companies or people owning a trademark. Before moving on I check if there are some other properties that retrieve such information. In order to do that I check which properties connects "Apple Inc." (wd:Q312) and its trademarks.

In [10]:
queryString = """
SELECT ?p ?pname COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270.
?t ?p wd:Q312.

?p <http://schema.org/name> ?pname.

}
ORDER BY DESC(?trademarks)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P176'), ('pname', 'manufacturer'), ('trademarks', '6')]
[('p', 'http://www.wikidata.org/prop/direct/P178'), ('pname', 'developer'), ('trademarks', '5')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by'), ('trademarks', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('pname', 'operator'), ('trademarks', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P749'), ('pname', 'parent organization'), ('trademarks', '1')]


5

I see that most trademarks are connected to Apple Inc. by means of the property "developed"(wdt:P178) and "manufacturer"(wdt:P176). I now check how many trademarks are connected with such properties.

In [11]:
queryString = """
SELECT COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270.
FILTER EXISTS{ ?t wdt:P178 ?d. }

}
"""

print("Results")
run_query(queryString)

Results
[('trademarks', '85')]


1

In [12]:
queryString = """
SELECT COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270.
FILTER EXISTS{ ?t wdt:P176 ?d. }

}
"""

print("Results")
run_query(queryString)

Results
[('trademarks', '1554')]


1

Since all properties connect some trademarks I try to use them all to assess which company or person own a trademark. I count how many trademarks I can connect to a company or person by means of these properties.

In [13]:
queryString = """
SELECT COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 


?t wdt:P31/wdt:P279* wd:Q167270;
    wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
    


}
"""

print("Results")
run_query(queryString)

Results
[('trademarks', '5525')]


1

I retrieved information about 5525 trademarks over 25105. In the end I check if trademarks identified with the property "owner of" (wdt:P1830) are already present in the result set above.

In [14]:
queryString = """
SELECT COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

{
    ?t wdt:P31/wdt:P279* wd:Q167270.
    ?s wdt:P1830 ?t.
}
MINUS{
    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
}    

}
"""

print("Results")
run_query(queryString)

Results
[('trademarks', '42')]


1

From this result I can assess the BGP to connect all trademarks with the company or person related to it is the following. 

In [18]:
queryString = """
SELECT COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

{
    ?t wdt:P31/wdt:P279* wd:Q167270.
    ?s wdt:P1830 ?t.
}
UNION{
    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
}    

}
"""

print("Results")
run_query(queryString)

Results
[('trademarks', '5567')]


1

Since I saw that I do not loose much information I will use the following query later on to retrieve information about companies and trademarks.

In [15]:
queryString = """
SELECT COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.  

}
"""

print("Results")
run_query(queryString)

Results
[('trademarks', '5525')]


1

## Types or categories of a trademark
This section contains the queries used to answer the second question: _Identify the BGP to retrieve types or categories for a given trademark_

I recall I retrieve these properties in the previous section:

| IRI           | Description   | Number of Trademarks   |
| -----------   | -----------   |-----------|
| `wdt:P136`    | genre    | 1951 | 
| `wdt:P452`    | industry   | 3730 |
| `wdt:P373`    | commons category   | 3511 |

Firstly, I check if all the properties retrieve the required information by returning the objects connected to a trademark by means of each property.

In [16]:
queryString = """
SELECT ?g ?genre COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P136 ?g.
    
    ?g <http://schema.org/name> ?genre.

}
GROUP BY ?g ?genre
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('g', 'http://www.wikidata.org/entity/Q8341'), ('genre', 'jazz'), ('trademarks', '791')]
[('g', 'http://www.wikidata.org/entity/Q38848'), ('genre', 'heavy metal'), ('trademarks', '316')]
[('g', 'http://www.wikidata.org/entity/Q11401'), ('genre', 'hip hop music'), ('trademarks', '196')]
[('g', 'http://www.wikidata.org/entity/Q37073'), ('genre', 'pop music'), ('trademarks', '76')]
[('g', 'http://www.wikidata.org/entity/Q11399'), ('genre', 'rock music'), ('trademarks', '72')]
[('g', 'http://www.wikidata.org/entity/Q3071'), ('genre', 'punk rock'), ('trademarks', '72')]
[('g', 'http://www.wikidata.org/entity/Q45981'), ('genre', 'rhythm and blues'), ('trademarks', '43')]
[('g', 'http://www.wikidata.org/entity/Q9778'), ('genre', 'electronic music'), ('trademarks', '42')]
[('g', 'http://www.wikidata.org/entity/Q9730'), ('genre', 'Western classical music'), ('trademarks', '41')]
[('g', 'http://www.wikidata.org/entity/Q56284716'), ('genre', 'various genre'), ('trademarks', '38')]
[('g',

25

Property "genre"(wdt:P136) is used for trademarks related to the music industry.

I now check the "industry" property(wdt:P452).

In [17]:
queryString = """
SELECT ?i ?industry COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452 ?i.
    
    ?i <http://schema.org/name> ?industry.

}
GROUP BY ?i ?industry
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('i', 'http://www.wikidata.org/entity/Q126793'), ('industry', 'retail'), ('trademarks', '795')]
[('i', 'http://www.wikidata.org/entity/Q190117'), ('industry', 'automotive industry'), ('trademarks', '504')]
[('i', 'http://www.wikidata.org/entity/Q581105'), ('industry', 'consumer electronics'), ('trademarks', '227')]
[('i', 'http://www.wikidata.org/entity/Q869095'), ('industry', 'brewing'), ('trademarks', '185')]
[('i', 'http://www.wikidata.org/entity/Q746359'), ('industry', 'music industry'), ('trademarks', '184')]
[('i', 'http://www.wikidata.org/entity/Q11707'), ('industry', 'restaurant'), ('trademarks', '174')]
[('i', 'http://www.wikidata.org/entity/Q171141'), ('industry', 'gastronomy'), ('trademarks', '124')]
[('i', 'http://www.wikidata.org/entity/Q11828862'), ('industry', 'clothing industry'), ('trademarks', '98')]
[('i', 'http://www.wikidata.org/entity/Q540912'), ('industry', 'food industry'), ('trademarks', '82')]
[('i', 'http://www.wikidata.org/entity/Q173799'), ('indust

25

Property "industry"(wdt:P452), as the name suggests, gives information about the industry each trademark belongs to.

In [18]:
queryString = """
SELECT ?cg COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P373 ?cg.

}
GROUP BY ?cg
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('cg', 'Organizations'), ('trademarks', '3')]
[('cg', 'Bacardi Rum'), ('trademarks', '2')]
[('cg', 'Cinema City'), ('trademarks', '2')]
[('cg', "Macy's"), ('trademarks', '2')]
[('cg', 'Avex Group'), ('trademarks', '2')]
[('cg', 'Nesquik'), ('trademarks', '2')]
[('cg', 'Radisson Hotels & Resorts'), ('trademarks', '2')]
[('cg', 'Atlantic Records'), ('trademarks', '2')]
[('cg', 'Albert Heijn'), ('trademarks', '2')]
[('cg', 'Point vert'), ('trademarks', '2')]
[('cg', 'Dr Pepper'), ('trademarks', '2')]
[('cg', 'Petit beurre'), ('trademarks', '2')]
[('cg', 'Shingen-mochi'), ('trademarks', '2')]
[('cg', 'ASUS'), ('trademarks', '2')]
[('cg', 'Kaiser-Kümmel'), ('trademarks', '1')]
[('cg', 'Chocolad Para'), ('trademarks', '1')]
[('cg', 'Calorie Mate'), ('trademarks', '1')]
[('cg', 'Dobro-guitars'), ('trademarks', '1')]
[('cg', 'Primark'), ('trademarks', '1')]
[('cg', 'StrataFlash'), ('trademarks', '1')]
[('cg', 'Pizza-La'), ('trademarks', '1')]
[('cg', 'Tzomet Sfarim'), ('trademarks', '

25

"Common Category"(wdt:P373) contains no useful information for this task. 

I now check if trademarks with "genre" information are contained in some type of industry.

In [19]:
queryString = """
SELECT ?i ?industry COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452 ?i.
    
    ?i <http://schema.org/name> ?industry.
        
    FILTER EXISTS { ?t wdt:P136 ?g. }

}
GROUP BY ?i ?industry
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('i', 'http://www.wikidata.org/entity/Q746359'), ('industry', 'music industry'), ('trademarks', '41')]
[('i', 'http://www.wikidata.org/entity/Q10302058'), ('industry', 'phonographic industry'), ('trademarks', '10')]
[('i', 'http://www.wikidata.org/entity/Q19595701'), ('industry', 'entertainment industry'), ('trademarks', '4')]
[('i', 'http://www.wikidata.org/entity/Q638'), ('industry', 'music'), ('trademarks', '4')]
[('i', 'http://www.wikidata.org/entity/Q625568'), ('industry', 'show business'), ('trademarks', '2')]
[('i', 'http://www.wikidata.org/entity/Q173799'), ('industry', 'entertainment'), ('trademarks', '2')]
[('i', 'http://www.wikidata.org/entity/Q39809'), ('industry', 'marketing'), ('trademarks', '1')]
[('i', 'http://www.wikidata.org/entity/Q18127'), ('industry', 'record label'), ('trademarks', '1')]
[('i', 'http://www.wikidata.org/entity/Q11650'), ('industry', 'electronics'), ('trademarks', '1')]
[('i', 'http://www.wikidata.org/entity/Q38848'), ('industry', 'heavy me

14

I check if all trademarks that have a genre also have the "industry" property.

In [20]:
queryString = """
SELECT COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P136 ?g;
        wdt:P452 ?i.        
        
}
"""

print("Results")
run_query(queryString)

Results
[('trademarks', '59')]


1

I check what are the genres of the trademarks where there is no property "industry"(wdt:P452).

In [21]:
queryString = """
SELECT ?g ?genre COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P136 ?g.
        
    ?g <http://schema.org/name> ?genre.   
        
    FILTER NOT EXISTS{ ?t wdt:P452 ?i. }
        
}
GROUP BY ?g ?genre
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('g', 'http://www.wikidata.org/entity/Q8341'), ('genre', 'jazz'), ('trademarks', '783')]
[('g', 'http://www.wikidata.org/entity/Q38848'), ('genre', 'heavy metal'), ('trademarks', '304')]
[('g', 'http://www.wikidata.org/entity/Q11401'), ('genre', 'hip hop music'), ('trademarks', '188')]
[('g', 'http://www.wikidata.org/entity/Q3071'), ('genre', 'punk rock'), ('trademarks', '71')]
[('g', 'http://www.wikidata.org/entity/Q37073'), ('genre', 'pop music'), ('trademarks', '67')]
[('g', 'http://www.wikidata.org/entity/Q11399'), ('genre', 'rock music'), ('trademarks', '66')]
[('g', 'http://www.wikidata.org/entity/Q9778'), ('genre', 'electronic music'), ('trademarks', '40')]
[('g', 'http://www.wikidata.org/entity/Q45981'), ('genre', 'rhythm and blues'), ('trademarks', '40')]
[('g', 'http://www.wikidata.org/entity/Q9730'), ('genre', 'Western classical music'), ('trademarks', '39')]
[('g', 'http://www.wikidata.org/entity/Q9759'), ('genre', 'blues'), ('trademarks', '37')]
[('g', 'http://www

25

From these results I can assess the BGP to retrieve the types of categories for a given trademark is the following.

In [22]:
queryString = """
SELECT DISTINCT ?t ?trademark ?c ?category
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452|wdt:P136 ?c.
    
    ?t <http://schema.org/name> ?trademark. 
    ?c <http://schema.org/name> ?category.   
        
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('t', 'http://www.wikidata.org/entity/Q16341017'), ('trademark', 'Famous Players'), ('c', 'http://www.wikidata.org/entity/Q20679540'), ('category', 'cinema chain')]
[('t', 'http://www.wikidata.org/entity/Q1761653'), ('trademark', 'Sogo'), ('c', 'http://www.wikidata.org/entity/Q126793'), ('category', 'retail')]
[('t', 'http://www.wikidata.org/entity/Q63335'), ('trademark', 'Foot Locker'), ('c', 'http://www.wikidata.org/entity/Q126793'), ('category', 'retail')]
[('t', 'http://www.wikidata.org/entity/Q71022064'), ('trademark', 'Footaction'), ('c', 'http://www.wikidata.org/entity/Q126793'), ('category', 'retail')]
[('t', 'http://www.wikidata.org/entity/Q7606770'), ('trademark', 'Stein Mart'), ('c', 'http://www.wikidata.org/entity/Q126793'), ('category', 'retail')]
[('t', 'http://www.wikidata.org/entity/Q2749082'), ('trademark', 'Lord & Taylor'), ('c', 'http://www.wikidata.org/entity/Q126793'), ('category', 'retail')]
[('t', 'http://www.wikidata.org/entity/Q3603767'), ('trademark',

25

## Companies with the largest number of trademarks
This section contains the queries used to answer the third question: _What companies have the largest number of trademarks?_ 

I recall I found the BGP to connect companies/people to their trademark in the first section therefore i can answer to the above question by means of the following query.

In [23]:
queryString = """
SELECT ?o ?company COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

{
    ?t wdt:P31/wdt:P279* wd:Q167270.
    ?o wdt:P1830 ?t.
    ?o <http://schema.org/name> ?company.
}
UNION{
    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
        
    ?o <http://schema.org/name> ?company.
}    
}
GROUP BY ?o ?company
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q38903'), ('company', 'Universal Music Group'), ('trademarks', '125')]
[('o', 'http://www.wikidata.org/entity/Q160746'), ('company', 'Nestlé'), ('trademarks', '84')]
[('o', 'http://www.wikidata.org/entity/Q3295867'), ('company', 'The Coca-Cola Company'), ('trademarks', '75')]
[('o', 'http://www.wikidata.org/entity/Q21077'), ('company', 'Warner Music Group'), ('trademarks', '75')]
[('o', 'http://www.wikidata.org/entity/Q812142'), ('company', 'Covestro'), ('trademarks', '57')]
[('o', 'http://www.wikidata.org/entity/Q334800'), ('company', 'PepsiCo'), ('trademarks', '51')]
[('o', 'http://www.wikidata.org/entity/Q157062'), ('company', 'Unilever'), ('trademarks', '45')]
[('o', 'http://www.wikidata.org/entity/Q12857502'), ('company', 'Mondelēz International'), ('trademarks', '42')]
[('o', 'http://www.wikidata.org/entity/Q327751'), ('company', 'Kraft Foods Group'), ('trademarks', '40')]
[('o', 'http://www.wikidata.org/entity/Q56760250'), ('company

25

I expected to see these companies in the results since they are the most famous ones. 

I recall I found the property "country" (wdt:P17) in the first section therefore I can also see which is the country with most trademarks.

In [24]:
queryString = """
SELECT ?c ?country COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P17 ?c.
    ?c <http://schema.org/name> ?country.
    
}
GROUP BY ?c ?country
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q30'), ('country', 'United States of America'), ('trademarks', '4527')]
[('c', 'http://www.wikidata.org/entity/Q145'), ('country', 'United Kingdom'), ('trademarks', '1470')]
[('c', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('trademarks', '1297')]
[('c', 'http://www.wikidata.org/entity/Q17'), ('country', 'Japan'), ('trademarks', '785')]
[('c', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy'), ('trademarks', '639')]
[('c', 'http://www.wikidata.org/entity/Q142'), ('country', 'France'), ('trademarks', '578')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('country', 'Brazil'), ('trademarks', '420')]
[('c', 'http://www.wikidata.org/entity/Q16'), ('country', 'Canada'), ('trademarks', '392')]
[('c', 'http://www.wikidata.org/entity/Q29'), ('country', 'Spain'), ('trademarks', '366')]
[('c', 'http://www.wikidata.org/entity/Q55'), ('country', 'Netherlands'), ('trademarks', '345')]
[('c', 'http://www.wikidata.org/enti

25

As expected USA is the country with most trademarks however we can notice Italy(wd:Q38) is the fifth country with most trademarks therefore I can return the companies with most trademarks in Italy. I expect to see "Ferrero SPA" in the result set.

In [25]:
queryString = """
SELECT ?o ?company COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o;
        wdt:P17 wd:Q38.

    ?o <http://schema.org/name> ?company.
    
}
GROUP BY ?o ?company
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q105614713'), ('company', 'Pam Panorama'), ('trademarks', '4')]
[('o', 'http://www.wikidata.org/entity/Q160746'), ('company', 'Nestlé'), ('trademarks', '3')]
[('o', 'http://www.wikidata.org/entity/Q85355'), ('company', 'Arnoldo Mondadori Editore'), ('trademarks', '3')]
[('o', 'http://www.wikidata.org/entity/Q21077'), ('company', 'Warner Music Group'), ('trademarks', '3')]
[('o', 'http://www.wikidata.org/entity/Q3954771'), ('company', 'Selex'), ('trademarks', '3')]
[('o', 'http://www.wikidata.org/entity/Q19616'), ('company', 'RAI'), ('trademarks', '2')]
[('o', 'http://www.wikidata.org/entity/Q3747293'), ('company', 'Fonit Cetra'), ('trademarks', '2')]
[('o', 'http://www.wikidata.org/entity/Q21190256'), ('company', 'Unes'), ('trademarks', '2')]
[('o', 'http://www.wikidata.org/entity/Q66912682'), ('company', 'La Feltrinelli'), ('trademarks', '2')]
[('o', 'http://www.wikidata.org/entity/Q105637594'), ('company', 'Net Global srl'), ('trademarks

25

The result set is a bit different than expected because the country information refers to the trademark not the company, therefore I repeat the last two queries referring to the company country.

In [26]:
queryString = """
SELECT ?c ?country COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
        
    ?o  wdt:P17 ?c.
    ?c <http://schema.org/name> ?country.
    
}
GROUP BY ?c ?country
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q30'), ('country', 'United States of America'), ('trademarks', '1668')]
[('c', 'http://www.wikidata.org/entity/Q31'), ('country', 'Belgium'), ('trademarks', '447')]
[('c', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('trademarks', '433')]
[('c', 'http://www.wikidata.org/entity/Q17'), ('country', 'Japan'), ('trademarks', '432')]
[('c', 'http://www.wikidata.org/entity/Q145'), ('country', 'United Kingdom'), ('trademarks', '369')]
[('c', 'http://www.wikidata.org/entity/Q142'), ('country', 'France'), ('trademarks', '234')]
[('c', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy'), ('trademarks', '145')]
[('c', 'http://www.wikidata.org/entity/Q55'), ('country', 'Netherlands'), ('trademarks', '141')]
[('c', 'http://www.wikidata.org/entity/Q39'), ('country', 'Switzerland'), ('trademarks', '107')]
[('c', 'http://www.wikidata.org/entity/Q148'), ('country', "People's Republic of China"), ('trademarks', '87')]
[('c', 'http

25

As far as countries with the highest number of trademarks the results are similar, however I see that now the second country is Belgium (wd:Q31) instead of United Kingdom. Therefore I decide to return the belgian companies with most trademarks.

In [27]:
queryString = """
SELECT ?company COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
        
    ?o wdt:P17 wd:Q31.
    
    ?o <http://schema.org/name> ?company.
    
}
GROUP BY ?company
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('company', 'De Proefbrouwerij'), ('trademarks', '28')]
[('company', 'Van Steenberghe brewery'), ('trademarks', '16')]
[('company', 'Brewery Strubbe'), ('trademarks', '11')]
[('company', 'Brasserie Du Bocq'), ('trademarks', '11')]
[('company', 'Anheuser-Busch InBev'), ('trademarks', '11')]
[('company', 'De Struise brewers'), ('trademarks', '10')]
[('company', 'Het Anker Brewery'), ('trademarks', '9')]
[('company', 'Pico brewery Alvinne'), ('trademarks', '8')]
[('company', 'Huyghe Brewery'), ('trademarks', '8')]
[('company', "'t Hofbrouwerijke"), ('trademarks', '7')]
[('company', 'Haacht'), ('trademarks', '7')]
[('company', 'Alken-Maes'), ('trademarks', '7')]
[('company', 'Van Honsebrouck Brewery'), ('trademarks', '6')]
[('company', 'Brasserie de Silly'), ('trademarks', '6')]
[('company', 'Brewery De Dochter van de Korenaar'), ('trademarks', '6')]
[('company', "Brasserie d'Écaussinnes"), ('trademarks', '5')]
[('company', 'Duvel Moortgat Brewery'), ('trademarks', '5')]
[('compan

25

Belgium is well known for its breweries and as I expect its high number of trademark is related to the beer industry. 

I can now retrieve the italian companies with the highest number of trademarks.

In [28]:
queryString = """
SELECT ?o ?company COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
        
    ?o wdt:P17 wd:Q38.
    
    ?o <http://schema.org/name> ?company.
    
}
GROUP BY ?o ?company
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q269792'), ('company', 'Ferrero SpA'), ('trademarks', '26')]
[('o', 'http://www.wikidata.org/entity/Q753685'), ('company', 'Campari Group'), ('trademarks', '8')]
[('o', 'http://www.wikidata.org/entity/Q673267'), ('company', 'Olivetti'), ('trademarks', '5')]
[('o', 'http://www.wikidata.org/entity/Q85355'), ('company', 'Arnoldo Mondadori Editore'), ('trademarks', '4')]
[('o', 'http://www.wikidata.org/entity/Q105614713'), ('company', 'Pam Panorama'), ('trademarks', '4')]
[('o', 'http://www.wikidata.org/entity/Q635484'), ('company', 'Armani'), ('trademarks', '3')]
[('o', 'http://www.wikidata.org/entity/Q27597'), ('company', 'Fiat'), ('trademarks', '3')]
[('o', 'http://www.wikidata.org/entity/Q3954771'), ('company', 'Selex'), ('trademarks', '3')]
[('o', 'http://www.wikidata.org/entity/Q105637594'), ('company', 'Net Global srl'), ('trademarks', '3')]
[('o', 'http://www.wikidata.org/entity/Q19616'), ('company', 'RAI'), ('trademarks', '2')]
[('o',

25

As I expected the italian company with the highest number of trademarks is Ferrero(wd:Q269792). I now return all the trademarks Ferrero owns.

In [29]:
queryString = """
SELECT DISTINCT ?t ?trademark
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 wd:Q269792.
    
    ?t <http://schema.org/name> ?trademark.
    
}
"""

print("Results")
run_query(queryString)

Results
[('t', 'http://www.wikidata.org/entity/Q1050530'), ('trademark', 'Tic Tac')]
[('t', 'http://www.wikidata.org/entity/Q1059233'), ('trademark', 'Kinder Chocolate')]
[('t', 'http://www.wikidata.org/entity/Q212193'), ('trademark', 'Nutella')]
[('t', 'http://www.wikidata.org/entity/Q2297653'), ('trademark', 'Kinder Bueno')]
[('t', 'http://www.wikidata.org/entity/Q1780501'), ('trademark', 'Raffaello')]
[('t', 'http://www.wikidata.org/entity/Q5433442'), ('trademark', 'Famous Amos')]
[('t', 'http://www.wikidata.org/entity/Q96402924'), ('trademark', 'Royal Dansk')]
[('t', 'http://www.wikidata.org/entity/Q5101733'), ('trademark', 'Chips Deluxe')]
[('t', 'http://www.wikidata.org/entity/Q3906863'), ('trademark', 'Pocket Coffee')]
[('t', 'http://www.wikidata.org/entity/Q1588515'), ('trademark', 'Mon Chéri')]
[('t', 'http://www.wikidata.org/entity/Q15975817'), ('trademark', 'Kinder Joy')]
[('t', 'http://www.wikidata.org/entity/Q181042'), ('trademark', 'Kinder Surprise')]
[('t', 'http://www.w

24

I recall I also found a property "inception"(wdt:P571) therefore I order Ferrero's trademarks based on the inception date (if available).

In [30]:
queryString = """
SELECT DISTINCT ?trademark ?year
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 wd:Q269792;
        wdt:P571 ?inception.
    
    BIND(year(?inception) AS ?year)
    
    ?t <http://schema.org/name> ?trademark.
    
}
ORDER BY ?year
"""

print("Results")
run_query(queryString)

Results
[('trademark', 'Thorntons Limited'), ('year', '1911')]
[('trademark', 'Tic Tac'), ('year', '1943')]
[('trademark', 'Mon Chéri'), ('year', '1956')]
[('trademark', 'Nutella'), ('year', '1964')]
[('trademark', 'Duplo'), ('year', '1964')]
[('trademark', 'Royal Dansk'), ('year', '1966')]
[('trademark', 'Kinder'), ('year', '1968')]
[('trademark', 'Yogurette'), ('year', '1970')]
[('trademark', 'Famous Amos'), ('year', '1975')]
[('trademark', 'Kinder Bueno'), ('year', '1990')]
[('trademark', 'Raffaello'), ('year', '1990')]
[('trademark', 'Kinder Joy'), ('year', '2001')]


12

The most ancient trademark is "Thorntons Limited" that is a british company bought by Ferrero in 2015, other than that the product with the oldest trademark is Tic Tac.

## Types of trademarks
This section contains the queries used to answer the forth question: _What are the types of trademarks, how many trademarks exist for each type?_

I recall I found the BGP to connect each trademark with each category in the second section. I can return the categories with the highest number of trademarks by means of the following query.

In [31]:
queryString = """
SELECT ?c ?category COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452|wdt:P136 ?c.
        
    ?c <http://schema.org/name> ?category.   
        
}
GROUP BY ?c ?category
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q126793'), ('category', 'retail'), ('trademarks', '795')]
[('c', 'http://www.wikidata.org/entity/Q8341'), ('category', 'jazz'), ('trademarks', '791')]
[('c', 'http://www.wikidata.org/entity/Q190117'), ('category', 'automotive industry'), ('trademarks', '504')]
[('c', 'http://www.wikidata.org/entity/Q38848'), ('category', 'heavy metal'), ('trademarks', '316')]
[('c', 'http://www.wikidata.org/entity/Q581105'), ('category', 'consumer electronics'), ('trademarks', '227')]
[('c', 'http://www.wikidata.org/entity/Q11401'), ('category', 'hip hop music'), ('trademarks', '196')]
[('c', 'http://www.wikidata.org/entity/Q869095'), ('category', 'brewing'), ('trademarks', '185')]
[('c', 'http://www.wikidata.org/entity/Q746359'), ('category', 'music industry'), ('trademarks', '184')]
[('c', 'http://www.wikidata.org/entity/Q11707'), ('category', 'restaurant'), ('trademarks', '174')]
[('c', 'http://www.wikidata.org/entity/Q171141'), ('category', 'gastronomy

25

Since the category with most trademarks is "retail"(wd:Q126793) I return the companies with most trademarks in such industry.

In [32]:
queryString = """
SELECT ?c ?company COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452 wd:Q126793;
         wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?c.
        
    ?c <http://schema.org/name> ?company.   
        
}
GROUP BY ?c ?company
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q2670967'), ('company', 'Casino Group'), ('trademarks', '7')]
[('c', 'http://www.wikidata.org/entity/Q909856'), ('company', 'Loblaw Companies'), ('trademarks', '6')]
[('c', 'http://www.wikidata.org/entity/Q153417'), ('company', 'Kroger'), ('trademarks', '6')]
[('c', 'http://www.wikidata.org/entity/Q3117359'), ('company', 'Carrefour'), ('trademarks', '5')]
[('c', 'http://www.wikidata.org/entity/Q483551'), ('company', 'Walmart'), ('trademarks', '4')]
[('c', 'http://www.wikidata.org/entity/Q1618373'), ('company', 'Coop Norge'), ('trademarks', '4')]
[('c', 'http://www.wikidata.org/entity/Q258748'), ('company', 'X5 Retail Group'), ('trademarks', '4')]
[('c', 'http://www.wikidata.org/entity/Q7512351'), ('company', 'Sigma Company Limited'), ('trademarks', '4')]
[('c', 'http://www.wikidata.org/entity/Q487494'), ('company', 'Tesco'), ('trademarks', '4')]
[('c', 'http://www.wikidata.org/entity/Q20539261'), ('company', 'Ahold Delhaize'), ('trademarks

25

I can also return the companies that have the highest number of trademarks in different categories.

In [33]:
queryString = """
SELECT ?o ?company COUNT(DISTINCT ?c) AS ?differentCategories
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452|wdt:P136 ?c;
         wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
        
    ?o <http://schema.org/name> ?company.   
        
}
GROUP BY ?o ?company
ORDER BY DESC(?differentCategories)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q38903'), ('company', 'Universal Music Group'), ('differentCategories', '49')]
[('o', 'http://www.wikidata.org/entity/Q21077'), ('company', 'Warner Music Group'), ('differentCategories', '47')]
[('o', 'http://www.wikidata.org/entity/Q219635'), ('company', 'BlackRock'), ('differentCategories', '23')]
[('o', 'http://www.wikidata.org/entity/Q56760250'), ('company', 'Sony Music'), ('differentCategories', '20')]
[('o', 'http://www.wikidata.org/entity/Q30431407'), ('company', 'FCMG'), ('differentCategories', '16')]
[('o', 'http://www.wikidata.org/entity/Q64627997'), ('company', 'David Russell Samuel'), ('differentCategories', '15')]
[('o', 'http://www.wikidata.org/entity/Q849363'), ('company', 'The Vanguard Group'), ('differentCategories', '15')]
[('o', 'http://www.wikidata.org/entity/Q30201990'), ('company', 'Pedro Fernando'), ('differentCategories', '15')]
[('o', 'http://www.wikidata.org/entity/Q183412'), ('company', 'EMI'), ('differentCategor

25

Since I am also considering the property "genre" that is related to the music industry the result set have a lot of record companies since it is reasonable they produce lots of different genres of music. Therefore I run the same query considering only the property "industry" (wdt:P452).

In [34]:
queryString = """
SELECT ?o ?company COUNT(DISTINCT ?c) AS ?differentCategories 
                GROUP_CONCAT(DISTINCT ?category; separator="; ") AS ?categories
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452 ?c;
         wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
    
    ?c <http://schema.org/name> ?category.
    ?o <http://schema.org/name> ?company.   
        
}
GROUP BY ?o ?company
ORDER BY DESC(?differentCategories)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q219635'), ('company', 'BlackRock'), ('differentCategories', '23'), ('categories', 'Accommodation and food service activities; International Standard Industrial Classification; big-box store; communication device; computer network; consumer electronics; digital distribution; discount store; electrical appliance; electronics; food and tobacco industry; food industry; foodservice; hardware store; information technology; mobile phone industry; networking hardware; petroleum industry; quick service restaurant sector; retail; retail chain; software industry; telecommunications industry')]
[('o', 'http://www.wikidata.org/entity/Q849363'), ('company', 'The Vanguard Group'), ('differentCategories', '15'), ('categories', 'Accommodation and food service activities; big-box store; consumer electronics; digital distribution; discount store; electronics; food and tobacco industry; foodservice; hardware store; information technology; mobile phone indust

10

In the end, I can return the top-20 categories with the highest number of trademarks owned by italian companies. I recall node "Italy" is wd:Q38 and property "country" is wdt:P17.

In [35]:
queryString = """
SELECT ?c ?category COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452|wdt:P136 ?c;
         wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
    
    ?o wdt:P17 wd:Q38.
    
    ?c <http://schema.org/name> ?category.  
        
}
GROUP BY ?c ?category
ORDER BY DESC(?trademarks)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q190117'), ('category', 'automotive industry'), ('trademarks', '8')]
[('c', 'http://www.wikidata.org/entity/Q25245117'), ('category', 'telecommunications industry'), ('trademarks', '6')]
[('c', 'http://www.wikidata.org/entity/Q3972943'), ('category', 'publishing'), ('trademarks', '6')]
[('c', 'http://www.wikidata.org/entity/Q126793'), ('category', 'retail'), ('trademarks', '6')]
[('c', 'http://www.wikidata.org/entity/Q837171'), ('category', 'financial services'), ('trademarks', '5')]
[('c', 'http://www.wikidata.org/entity/Q1416657'), ('category', 'Finance'), ('trademarks', '4')]
[('c', 'http://www.wikidata.org/entity/Q540912'), ('category', 'food industry'), ('trademarks', '4')]
[('c', 'http://www.wikidata.org/entity/Q806718'), ('category', 'banking industry'), ('trademarks', '3')]
[('c', 'http://www.wikidata.org/entity/Q178512'), ('category', 'public transport'), ('trademarks', '3')]
[('c', 'http://www.wikidata.org/entity/Q331814'), ('cat

20

Since the trademarks are not as many for each category I can retrieve them for the first 10 categories.

In [36]:
queryString = """
SELECT ?category COUNT(DISTINCT ?t) AS ?numTMs 
                    GROUP_CONCAT(DISTINCT ?trademark; separator="; ") AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452|wdt:P136 ?c;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
    
    ?o wdt:P17 wd:Q38.
    
    ?t <http://schema.org/name> ?trademark.
    ?c <http://schema.org/name> ?category.  
        
}
GROUP BY ?category
ORDER BY DESC(?numTMs)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('category', 'automotive industry'), ('numTMs', '8'), ('trademarks', 'Aermacchi; CNH Industrial; Fiat Brevetti; Imperial; Innocenti; Landini; Laverda; Pasquali vehicles')]
[('category', 'telecommunications industry'), ('numTMs', '6'), ('trademarks', 'FiberCop; Italtel; Kena Mobile; PosteMobile; Rai Teche; Very Mobile')]
[('category', 'retail'), ('numTMs', '6'), ('trademarks', 'Conad City; Esselunga; Intimissimi; Segafredo Zanetti; U!; U2')]
[('category', 'publishing'), ('numTMs', '6'), ('trademarks', 'Academic Press; Caltagirone Editore; Garzanti; La Feltrinelli; Longanesi; Orbis Publishing Ltd.')]
[('category', 'financial services'), ('numTMs', '4'), ('trademarks', 'Banca Carim; Banca Intermobiliare; Bank of Sicily; Credito Italiano')]
[('category', 'food industry'), ('numTMs', '4'), ('trademarks', 'Mulino Bianco; Thorntons Limited; Voiello; Wasabröd')]
[('category', 'banking industry'), ('numTMs', '4'), ('trademarks', 'Bank of Sicily; Carisbo; HypoVereinsbank; UniCredit Bank

10

## Number of trademarks across types, companies and countries
This section contains the queries used to answer the last question: _Analyze the number of trademarks across types, companies, and countries_

### Companies or People
Firstly I need to understand how to distinguish between companies holding trademarks and people. As a start I can return the instance of (wdt:P31) of the objects connected to the trademarks.

In [37]:
queryString = """
SELECT ?i ?instanceOf COUNT(DISTINCT ?o) AS ?objs
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
        
    ?o  wdt:P31 ?i.
    
    ?i <http://schema.org/name> ?instanceOf.
    
}
GROUP BY ?i ?instanceOf
ORDER BY DESC(?objs)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('i', 'http://www.wikidata.org/entity/Q4830453'), ('instanceOf', 'business'), ('objs', '1541')]
[('i', 'http://www.wikidata.org/entity/Q6881511'), ('instanceOf', 'enterprise'), ('objs', '667')]
[('i', 'http://www.wikidata.org/entity/Q891723'), ('instanceOf', 'public company'), ('objs', '573')]
[('i', 'http://www.wikidata.org/entity/Q5'), ('instanceOf', 'human'), ('objs', '247')]
[('i', 'http://www.wikidata.org/entity/Q18127'), ('instanceOf', 'record label'), ('objs', '224')]
[('i', 'http://www.wikidata.org/entity/Q131734'), ('instanceOf', 'brewery'), ('objs', '206')]
[('i', 'http://www.wikidata.org/entity/Q2085381'), ('instanceOf', 'publisher'), ('objs', '109')]
[('i', 'http://www.wikidata.org/entity/Q2442401'), ('instanceOf', 'record company'), ('objs', '107')]
[('i', 'http://www.wikidata.org/entity/Q786820'), ('instanceOf', 'automobile manufacturer'), ('objs', '81')]
[('i', 'http://www.wikidata.org/entity/Q783794'), ('instanceOf', 'company'), ('objs', '73')]
[('i', 'http://w

25

I can assess that in order to retrieve companies I can filter out instances of "human"(wd:Q5) while when I want to retrieve people connected to trademarks I filter only results that are instance of "human".

### US Companies
_How many U.S. companies hold trademarks? In which sectors?_

As I saw earlier the node "USA" is wd:Q30 therefore I can return the total number of trademarks held by american companies as a start.

In [38]:
queryString = """
SELECT COUNT(DISTINCT ?t) AS ?UStrademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o.
        
    ?o  wdt:P17 wd:Q30;
        wdt:P31 ?i.
    
    FILTER(?i != wd:Q5).
    
}
"""

print("Results")
run_query(queryString)

Results
[('UStrademarks', '1668')]


1

Now I return the categories with the highest number of trademarks held by american companies.

In [39]:
queryString = """
SELECT ?c ?category COUNT(DISTINCT ?t) AS ?UStrademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?o;
        wdt:P452|wdt:P136 ?c.
        
    ?o  wdt:P17 wd:Q30;
        wdt:P31 ?i.
    
    ?c <http://schema.org/name> ?category.
    
    FILTER(?i != wd:Q5).
    
}
GROUP BY ?c ?category
ORDER BY DESC(?UStrademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q126793'), ('category', 'retail'), ('UStrademarks', '65')]
[('c', 'http://www.wikidata.org/entity/Q11401'), ('category', 'hip hop music'), ('UStrademarks', '35')]
[('c', 'http://www.wikidata.org/entity/Q8341'), ('category', 'jazz'), ('UStrademarks', '34')]
[('c', 'http://www.wikidata.org/entity/Q746359'), ('category', 'music industry'), ('UStrademarks', '28')]
[('c', 'http://www.wikidata.org/entity/Q1495452'), ('category', 'hospitality industry'), ('UStrademarks', '20')]
[('c', 'http://www.wikidata.org/entity/Q56284716'), ('category', 'various genre'), ('UStrademarks', '18')]
[('c', 'http://www.wikidata.org/entity/Q540912'), ('category', 'food industry'), ('UStrademarks', '18')]
[('c', 'http://www.wikidata.org/entity/Q45981'), ('category', 'rhythm and blues'), ('UStrademarks', '17')]
[('c', 'http://www.wikidata.org/entity/Q38848'), ('category', 'heavy metal'), ('UStrademarks', '15')]
[('c', 'http://www.wikidata.org/entity/Q190117'), ('cate

25

I can also retrieve the american companies with the highest number of trademarks.

In [40]:
queryString = """
SELECT ?c ?company COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?c.
        
    ?c  wdt:P17 wd:Q30;
        wdt:P31 ?i.
    
    ?c <http://schema.org/name> ?company.
    
    FILTER(?i != wd:Q5).
    
}
GROUP BY ?c ?company
ORDER BY DESC(?trademarks)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q38903'), ('company', 'Universal Music Group'), ('trademarks', '125')]
[('c', 'http://www.wikidata.org/entity/Q3295867'), ('company', 'The Coca-Cola Company'), ('trademarks', '75')]
[('c', 'http://www.wikidata.org/entity/Q21077'), ('company', 'Warner Music Group'), ('trademarks', '74')]
[('c', 'http://www.wikidata.org/entity/Q334800'), ('company', 'PepsiCo'), ('trademarks', '50')]
[('c', 'http://www.wikidata.org/entity/Q12857502'), ('company', 'Mondelēz International'), ('trademarks', '42')]
[('c', 'http://www.wikidata.org/entity/Q56760250'), ('company', 'Sony Music'), ('trademarks', '38')]
[('c', 'http://www.wikidata.org/entity/Q327751'), ('company', 'Kraft Foods Group'), ('trademarks', '38')]
[('c', 'http://www.wikidata.org/entity/Q212405'), ('company', 'Procter & Gamble'), ('trademarks', '32')]
[('c', 'http://www.wikidata.org/entity/Q1056637'), ('company', 'The Hershey Company'), ('trademarks', '32')]
[('c', 'http://www.wikidata.org/ent

15

Since the first company produces films and musics I retrieve the categories of "The Coca-Cola Company"(wd:Q3295867) trademarks.

In [41]:
queryString = """
SELECT ?c ?category COUNT(DISTINCT ?t) AS ?CCtrademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 wd:Q3295867;
        wdt:P452|wdt:P136 ?c.
    
    ?c <http://schema.org/name> ?category.
    
}
GROUP BY ?c ?category
ORDER BY DESC(?CCtrademarks)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q4899370'), ('category', 'beverage industry'), ('CCtrademarks', '4')]
[('c', 'http://www.wikidata.org/entity/Q171141'), ('category', 'gastronomy'), ('CCtrademarks', '1')]


2

There are 75 trademarks owned by Coca-Cola howevere I retrieved only 5 of the with this query. I return all coca-cola's trademarks properties to see if I can find the category information elsewhere.

In [42]:
queryString = """
SELECT ?p ?pname COUNT(DISTINCT ?t) AS ?CCtrademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 wd:Q3295867;
        ?p ?o.
    
    ?p <http://schema.org/name> ?pname.
    
}
GROUP BY ?p ?pname
ORDER BY DESC(?CCtrademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('CCtrademarks', '75')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pname', 'Freebase ID'), ('CCtrademarks', '71')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pname', 'inception'), ('CCtrademarks', '63')]
[('p', 'http://www.wikidata.org/prop/direct/P176'), ('pname', 'manufacturer'), ('CCtrademarks', '59')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin'), ('CCtrademarks', '51')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('pname', 'official website'), ('CCtrademarks', '33')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pname', 'Commons category'), ('CCtrademarks', '27')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image'), ('CCtrademarks', '23')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('CCtrademarks', '18')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pname', 'logo

25

I assess I miss the type information however I can see the most ancient trademarks Coca-Cola owns with the property "inception"(wdt:P571).

In [43]:
queryString = """
SELECT DISTINCT ?trademark ?inception
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 wd:Q3295867;
        wdt:P571 ?inception.
        
    ?t <http://schema.org/name> ?trademark.
    
}
ORDER BY ?inception
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('trademark', "Barq's"), ('inception', '1842-01-01T00:00:00Z')]
[('trademark', 'Coca-Cola'), ('inception', '1886-05-08T00:00:00Z')]
[('trademark', 'Kia-Ora'), ('inception', '1903-01-01T00:00:00Z')]
[('trademark', 'Fanta'), ('inception', '1940-01-01T00:00:00Z')]
[('trademark', 'Minute Maid'), ('inception', '1945-01-01T00:00:00Z')]
[('trademark', 'Gold Spot'), ('inception', '1952-01-01T00:00:00Z')]
[('trademark', 'Sprite'), ('inception', '1961-01-01T00:00:00Z')]
[('trademark', 'Tab'), ('inception', '1963-01-01T00:00:00Z')]
[('trademark', 'Fresca'), ('inception', '1966-01-01T00:00:00Z')]
[('trademark', 'Leed'), ('inception', '1967-01-01T00:00:00Z')]
[('trademark', 'Costa Coffee'), ('inception', '1971-01-01T00:00:00Z')]
[('trademark', 'Pibb Xtra'), ('inception', '1972-01-01T00:00:00Z')]
[('trademark', 'Lilt'), ('inception', '1975-01-01T00:00:00Z')]
[('trademark', 'Iron Brew'), ('inception', '1975-01-01T00:00:00Z')]
[('trademark', 'Maaza'), ('inception', '1976-01-01T00:00:00Z')]
[(

25

### People connected to trademarks
_How many people hold or are connected to trademarks? In which role?_

I recall I can filter the owners of a trademark that are instance of "human"(wd:Q5). Firstly I return the number of trademarks held by a person.

In [44]:
queryString = """
SELECT COUNT(DISTINCT ?t) AS ?peopleTrademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?p.
    
    ?p wdt:P31 wd:Q5.
    
}
"""

print("Results")
run_query(queryString)

Results
[('peopleTrademarks', '223')]


1

I return such people properties in order to see how to retrieve the information about their role.

In [45]:
queryString = """
SELECT ?prop ?pname COUNT(DISTINCT ?p) AS ?people
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?p.
    
    ?p wdt:P31 wd:Q5;
        ?prop ?obj.
        
    ?prop <http://schema.org/name> ?pname.
    
}
GROUP BY ?prop ?pname
ORDER BY DESC(?people)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('people', '739')]
[('prop', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name'), ('people', '391')]
[('prop', 'http://www.wikidata.org/prop/direct/P7859'), ('pname', 'WorldCat Identities ID'), ('people', '284')]
[('prop', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender'), ('people', '251')]
[('prop', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth'), ('people', '226')]
[('prop', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed'), ('people', '205')]
[('prop', 'http://www.wikidata.org/prop/direct/P1830'), ('pname', 'owner of'), ('people', '182')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image'), ('people', '156')]
[('prop', 'http://www.wikidata.org/prop/direct/P734'), ('pname', 'family name'), ('people', '145')]
[('prop', 'http://www.wikidata.org/prop/direct/P937'), ('pname'

25

I can return who is the owner of the highest number of trademarks by means of the property "owner of" (wdt:P1830).

In [46]:
queryString = """
SELECT ?p ?person COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270.
    
    ?p wdt:P31 wd:Q5;
       wdt:P1830 ?t.
        
    ?p <http://schema.org/name> ?person.
    
}
GROUP BY ?p ?person
ORDER BY DESC(?trademarks)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q30201990'), ('person', 'Pedro Fernando'), ('trademarks', '6')]
[('p', 'http://www.wikidata.org/entity/Q3170637'), ('person', 'Jean Baud'), ('trademarks', '2')]
[('p', 'http://www.wikidata.org/entity/Q105455093'), ('person', 'Sadoc Vazkez'), ('trademarks', '2')]
[('p', 'http://www.wikidata.org/entity/Q6524850'), ('person', 'Leon René'), ('trademarks', '2')]
[('p', 'http://www.wikidata.org/entity/Q62766'), ('person', 'JAY-Z'), ('trademarks', '2')]
[('p', 'http://www.wikidata.org/entity/Q65551875'), ('person', 'Aki Roukala'), ('trademarks', '2')]
[('p', 'http://www.wikidata.org/entity/Q61249179'), ('person', 'Alexander Julien'), ('trademarks', '2')]
[('p', 'http://www.wikidata.org/entity/Q7108819'), ('person', 'Otis René'), ('trademarks', '2')]
[('p', 'http://www.wikidata.org/entity/Q105961166'), ('person', 'Jean-Marie Mazard'), ('trademarks', '1')]
[('p', 'http://www.wikidata.org/entity/Q289647'), ('person', 'David Ellefson'), ('trademarks'

15

Since the number of trademarks is managable I can also concatenate their trademarks.

In [47]:
queryString = """
SELECT ?p ?person COUNT(DISTINCT ?t) AS ?numTMs 
                  GROUP_CONCAT(DISTINCT ?trademark; separator="; ") AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270.
    
    ?p wdt:P31 wd:Q5;
       wdt:P1830 ?t.
        
    ?p <http://schema.org/name> ?person.
    ?t <http://schema.org/name> ?trademark
    
}
GROUP BY ?p ?person
ORDER BY DESC(?numTMs)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q30201990'), ('person', 'Pedro Fernando'), ('numTMs', '6'), ('trademarks', 'B.C Rappers; Coca Overdose Kaizen Entertainment; FCMG; Lo-fi Worldwide; TDN Entertainment; TVY Entertainment')]
[('p', 'http://www.wikidata.org/entity/Q3170637'), ('person', 'Jean Baud'), ('numTMs', '2'), ('trademarks', 'Franprix; Leader Price')]
[('p', 'http://www.wikidata.org/entity/Q105455093'), ('person', 'Sadoc Vazkez'), ('numTMs', '2'), ('trademarks', 'Sadoc Vazkez Music Records, Inc.; Universal Academy Records')]
[('p', 'http://www.wikidata.org/entity/Q6524850'), ('person', 'Leon René'), ('numTMs', '2'), ('trademarks', 'Excelsior Records; Exclusive')]
[('p', 'http://www.wikidata.org/entity/Q62766'), ('person', 'JAY-Z'), ('numTMs', '2'), ('trademarks', '40/40 Club; Roc Nation')]
[('p', 'http://www.wikidata.org/entity/Q65551875'), ('person', 'Aki Roukala'), ('numTMs', '2'), ('trademarks', 'Is This Art!; Kioski')]
[('p', 'http://www.wikidata.org/entity/Q6124917

15

I notice most of them owns company records or record labels. I can also count how many trademarks are related to male and female people by means of the property "sex or gender"(wdt:P21).

In [48]:
queryString = """
SELECT ?gender COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?p.
    
    ?p wdt:P31 wd:Q5;
       wdt:P21 ?g.
       
    ?g <http://schema.org/name> ?gender.
    
}
GROUP BY ?gender
ORDER BY DESC(?trademarks)
"""

print("Results")
run_query(queryString)

Results
[('gender', 'male'), ('trademarks', '10093')]
[('gender', 'female'), ('trademarks', '1284')]


2

As expected female trademark owners are 1/10 with respect to male trademark owners.

In the end I can return the country of citizenship of the people who has the highest number of trademarks by means of the property "country of citizenship" (wdt:P27).

In [49]:
queryString = """
SELECT ?c ?country COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P127|wdt:P749|wdt:P178|wdt:P176 ?p.
    
    ?p wdt:P31 wd:Q5;
       wdt:P27 ?c.
       
    ?c <http://schema.org/name> ?country.
    
}
GROUP BY ?c ?country
ORDER BY DESC(?trademarks)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q29'), ('country', 'Spain'), ('trademarks', '1720')]
[('c', 'http://www.wikidata.org/entity/Q30'), ('country', 'United States of America'), ('trademarks', '1449')]
[('c', 'http://www.wikidata.org/entity/Q31'), ('country', 'Belgium'), ('trademarks', '1028')]
[('c', 'http://www.wikidata.org/entity/Q29999'), ('country', 'Kingdom of the Netherlands'), ('trademarks', '967')]
[('c', 'http://www.wikidata.org/entity/Q142'), ('country', 'France'), ('trademarks', '784')]
[('c', 'http://www.wikidata.org/entity/Q174193'), ('country', 'United Kingdom of Great Britain and Ireland'), ('trademarks', '463')]
[('c', 'http://www.wikidata.org/entity/Q55'), ('country', 'Netherlands'), ('trademarks', '345')]
[('c', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('trademarks', '315')]
[('c', 'http://www.wikidata.org/entity/Q34'), ('country', 'Sweden'), ('trademarks', '310')]
[('c', 'http://www.wikidata.org/entity/Q145'), ('country', 'United King

15

Unexpectedly Spain(wd:Q29) is the country of citizenships with most trademarks related to people.

### Sector with the highest number of trademarks
_In which sector there is the highest number of trademarks?_

I recall I found the BGP to connect each trademark with each category in the second section. I can return the categories with the highest number of trademarks.

In [50]:
queryString = """
SELECT ?c ?category COUNT(DISTINCT ?t) AS ?trademarks
WHERE { 

    ?t wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452|wdt:P136 ?c.
        
    ?c <http://schema.org/name> ?category.   
        
}
GROUP BY ?c ?category
ORDER BY DESC(?trademarks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q126793'), ('category', 'retail'), ('trademarks', '795')]
[('c', 'http://www.wikidata.org/entity/Q8341'), ('category', 'jazz'), ('trademarks', '791')]
[('c', 'http://www.wikidata.org/entity/Q190117'), ('category', 'automotive industry'), ('trademarks', '504')]
[('c', 'http://www.wikidata.org/entity/Q38848'), ('category', 'heavy metal'), ('trademarks', '316')]
[('c', 'http://www.wikidata.org/entity/Q581105'), ('category', 'consumer electronics'), ('trademarks', '227')]
[('c', 'http://www.wikidata.org/entity/Q11401'), ('category', 'hip hop music'), ('trademarks', '196')]
[('c', 'http://www.wikidata.org/entity/Q869095'), ('category', 'brewing'), ('trademarks', '185')]
[('c', 'http://www.wikidata.org/entity/Q746359'), ('category', 'music industry'), ('trademarks', '184')]
[('c', 'http://www.wikidata.org/entity/Q11707'), ('category', 'restaurant'), ('trademarks', '174')]
[('c', 'http://www.wikidata.org/entity/Q171141'), ('category', 'gastronomy

25